# High-level Keras (TF) Example

In [1]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "tensorflow"
import keras as K
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
from common.params import *
from common.utils import *

Using TensorFlow backend.
/opt/conda/envs/py3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# channels_last is faster
K.backend.set_image_data_format('channels_last')

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tensorflow.__version__)
print(K.backend.backend())
# Channels should be last (otherwise slow)
print(K.backend.image_data_format())
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.6.3 |Anaconda, Inc.| (default, Nov 20 2017, 20:41:42) 
[GCC 7.2.0]
Keras:  2.1.1
Numpy:  1.13.3
Tensorflow:  1.4.0
tensorflow
channels_last
GPU:  ['Tesla K80']


In [4]:
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'

In [5]:
def create_symbol():
    model = Sequential()
    
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(N_CLASSES, activation='softmax'))
    return model

In [6]:
def init_model(m):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(LR, MOMENTUM),
        metrics = ['accuracy'])
    return m

In [7]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=False, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)
float32 float32 int32 int32
CPU times: user 872 ms, sys: 536 ms, total: 1.41 s
Wall time: 1.41 s


In [8]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 180 ms, sys: 0 ns, total: 180 ms
Wall time: 179 ms


In [9]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 41.9 ms, sys: 494 µs, total: 42.4 ms
Wall time: 41.8 ms


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 50)        1400      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 50)        22550     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 50)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 50)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 100)       45100     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 100)       90100     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 100)         0         
__________

In [11]:
%%time
# 241s
# Train model
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/10
50000/50000 [==============================] - 27s 545us/step - loss: 1.8061 - acc: 0.3359
Epoch 2/10
50000/50000 [==============================] - 25s 504us/step - loss: 1.3784 - acc: 0.5007
Epoch 3/10
50000/50000 [==============================] - 25s 504us/step - loss: 1.1779 - acc: 0.5773
Epoch 4/10
50000/50000 [==============================] - 25s 506us/step - loss: 1.0268 - acc: 0.6345
Epoch 5/10
50000/50000 [==============================] - 25s 504us/step - loss: 0.9070 - acc: 0.6789
Epoch 6/10
50000/50000 [==============================] - 25s 509us/step - loss: 0.8193 - acc: 0.7117
Epoch 7/10
50000/50000 [==============================] - 25s 508us/step - loss: 0.7423 - acc: 0.7367
Epoch 8/10
50000/50000 [==============================] - 25s 509us/step - loss: 0.6810 - acc: 0.7596
Epoch 9/10
50000/50000 [==============================] - 26s 510us/step - loss: 0.6241 - acc: 0.7787
Epoch 10/10
50000/50000 [==============================] - 25s 510us/step - loss: 

In [12]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 563 ms, sys: 151 ms, total: 714 ms
Wall time: 1.52 s


In [13]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.7702
